In [ ]:
"""
Working with the database.

This notebook show you how to fetch records from the database.
The records loaded from spreadsheet rows or returned from the eBird API
are split across five different models: Checklist, Location, Observer,
Observation and Species, which saves a lot of space. However this means 
that database queries must join the tables to access all the fields. SQLite 
is extremely fast at doing so the overhead is minimal. The query examples 
in this notebook show you have to join the tables to return complete 
objects (Checklist, etc.) or just selected fields.

"""
from sqlalchemy import create_engine, select
from sqlalchemy.orm import Session


from ebird.notebooks import settings
from ebird.notebooks.models import Checklist, Location, Observation, Observer, Species

db_dir = settings.DATABASE_DIR
db_name = "api_checklists"
db_url = f"sqlite+pysqlite:///{db_dir}/{db_name}.sqlite3"

engine = create_engine(db_url)

In [ ]:
# Load selected fields for each observation.
with Session(engine) as session:
    rows = session.execute(
        select(Observation.count, Species.common_name, Checklist.date, Checklist.time, Checklist.identifier, Location.name, Observer.name)
        .join(Observation.species)
        .join(Observation.checklist)
        .join(Observation.location)
        .join(Observation.observer)
        .order_by(Checklist.identifier)
        .limit(10)
    )
    for row in rows:
        print(row)

In [ ]:
# Load all the information on each checklist, returning the fields as objects. 
with Session(engine) as session:
    rows = session.execute(
        select(Checklist, Location, Observer)
        .join(Location)
        .join(Observer)
        .limit(10)
    )
    for row in rows:
        print(row.Checklist.identifier, row.Checklist.date, row.Checklist.time, row.Location.name, row.Observer.name)